In [ ]:
import base64
import datetime
import json
import operator
from urllib.parse import urlencode

import requests

In [ ]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"

    
    def __init__(self,client_id,client_secret, *args, **kwargs):
        super().__init__(*args,**kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret.")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())

        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}" 
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200,299):
            raise Exception("could not authenticate client.")
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] #seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    
    def get_track(self, _id):
        return self.get_resource(_id, resource_type="tracks")
    
    def get_audio_features(self, _id):
        return self.get_resource(_id, resource_type='audio-features')
    
    def get_audio_analysis(self, _id):
        return self.get_resource(_id, resource_type='audio-analysis')
    
    def get_user_profile(self, _id):
        return self.get_resource(_id, resource_type="users")
    
    def get_user_playlists(self, _id):
        return self.get_resource(_id + "/playlists", resource_type="users")
    
    def get_playlist_items(self, _id):
        return self.get_resource(_id + "/tracks", resource_type="playlists")
    
    def get_my_profile(self):
        endpoint = "https://api.spotify/com/v1/me"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def base_search(self, query_params):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type="artist"): 
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}:" for k,v in query.items()])
        if operator != None and operator_query != None:
            #gives the option to set operator to NOT or OR to modify search
            #eg. "track": "Danger", "operator" = 'NOT', operator_query = "Zone"
                #displays top tracks that have "Danger" and not "zone" in track title
            if operator.lower() == "or" or operator.lower() =="not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        
        return self.base_search(query_params)
    
    def search_track_id(self, track_name, artist_name=None):
        #searches for song name on spotify
        search_results = self.search(query=track_name, search_type = "track")
                
        if len(search_results) == 0:
            return None
        
        if 'tracks' in search_results.keys():
            if artist_name is not None:
                #print(search_results)
                for i in range(len(search_results)):
                    #print("iterations: " + str(i))
                    current_option = search_results['tracks']['items'][i]
                    if track_name.lower() in current_option['name'].lower():
                        print("current_option['album']['artist'][0]['name']: " + current_option['album']['artists'][0]['name'])

                        if artist_name.lower() in current_option['album']['artists'][0]['name'].lower():
                            track_id = current_option['id']

                            return track_id
                        
            #if no artist name is given, first result in search will be taken
            track_id = search_results['tracks']['items'][0]['id']
            return track_id
        return None
    

class Track(object):

    def __init__(self, _id, client_id, client_secret):
        spotify = SpotifyAPI(client_id, client_secret)
        self.id = _id
        if 'name' in (spotify.get_track(_id)).keys():
            self.track = (spotify.get_track(_id))['name']
        if 'album' in (spotify.get_track(_id)).keys():
            self.album = (spotify.get_track(_id))['album']['name']
        if 'artist' in (spotify.get_track(_id)).keys():
            self.artist = (spotify.get_track(_id))['artists'][0]['name']
        self.audio_features = spotify.get_audio_features(_id)
    
class UserPlaylists(object):
    
    def __init__(self, _id, client_id, client_secret):
        spotify = SpotifyAPI(client_id, client_secret)
        #note: _id is the user id
        self.user_id = _id
        self.playlists = []
        for i in spotify.get_user_playlists(_id)['items']:
            self.playlists.append(i['name'])
        
    def playlist_tracks(self, playlist_name):
        '''
        Returns a json-style list of dictionaries each containing a track
        name + artist name
        '''
        user_playlists = spotify.get_user_playlists(self.user_id)
        for playlist in user_playlists['items']:
            if playlist_name.lower() in (playlist['name']).lower():
                playlist_id = playlist['id']
                playlist_tracks = []
                #print(playlist)
                for track in spotify.get_playlist_items(playlist_id)['items']:
                    
                    #print(track['track']['name'])
                    track_info = {}
                    track_info['id'] = track['track']['id']
                    track_info['trackName'] = track['track']['name']
                    track_info['artistName'] = track['track']['artists'][0]['name']
                    
                    playlist_tracks.append(track_info)
            
                return playlist_tracks
            
    def playlist_audio_features(self, playlist_name):
        '''
        (list of dictionaries) -> (list of dictionaries)
        Takes a playlist and creates a dictionary
        with all the attributes as keywords and a list of the corresponding
        values for each song.  
        '''
        track_list = self.playlist_tracks(playlist_name)
        audio_features_dict = {}
        for track in track_list:
            audio_features = spotify.get_audio_features(track['id'])
            for feature in audio_features:
                if feature != "id" and feature != 'uri' and feature != 'type' and feature != 'track_href' and feature != "analysis_url":
                    if feature not in audio_features_dict.keys():
                        audio_features_dict[feature] = [audio_features[feature]]
                    else:
                        new_value = audio_features_dict[feature] + [audio_features[feature]]
                        audio_features_dict[feature] = new_value
                
        return audio_features_dict

In [ ]:
def ExtractAudioFeatures(track_list, client_id, client_secret):
    '''
    (list of dictionaries) -> (list of dictionaries)
    Takes a spotify Streaming History and creates a dictionary
    with all the attributes as keywords and a list of the corresponding
    values for each song.
    '''
    spotify = SpotifyAPI(client_id, client_secret)
    audio_features_dict = {}
    
    for track_info in track_list:
        track = Track(spotify.search_track_id(track_info["trackName"], track_info["artistName"]), client_id, client_secret)
        for feature in track.audio_features:
            if feature not in audio_features_dict:
                audio_features_dict[feature] = [track.audio_features[feature]]
            else: 
                new_value = audio_features_dict[feature] + [track.audio_features[feature]]
                audio_features_dict[feature] = new_value
    return audio_features_dict

def DisplayMenu():
    '''
    Shows program title and gives a list of commands that can be activated
    '''
    print("     SPOTIFY DATA PROGRAM")
    print("0 - Quit\n"
          + "1 - Upload Data\n"
          + "2 - Parse Track Attributes & Logistic Regression\n"
          + "3 - djaflkjd")
    
def average_attributes(attribute_list):
    
    average_feature_attributes = {}
    
    for feature in attribute_list:
        average = sum(attribute_list[feature]) / len(attribute_list[feature])
        average_feature_attributes[feature] = average
        
    return average_feature_attributes

In [ ]:
#insert client info

client_id = "2fda0276fd834b4cb38fbca7805ce17c"
client_secret = "43c8a41a3c884c3b9e188e36ab353de0"

In [ ]:
#create an API object
spotify = SpotifyAPI(client_id, client_secret)

In [ ]:
f1, f2 = open('StreamingHistory1.json'), open('StreamingHistory0.json')
data1, data2 = json.load(f1), json.load(f2)
print(data1)

In [ ]:
#track1 = Track(spotify.search_track_id("Ambitionz", "pac"), client_id, client_secret)
#print(track1.track)

In [ ]:
#audio_feats = ExtractAudioFeatures(data2, client_id, client_secret)

#print("Audio features:")
#print(audio_feats)

In [ ]:
my_user = spotify.get_user_playlists('gabrielescobar1')
print(my_user)

In [ ]:
my_playlists = UserPlaylists('gabrielescobar1', client_id,client_secret)


In [ ]:
my_playlists= UserPlaylists('gabrielescobar1', client_id, client_secret)
print(spotify.get_user_playlists('gabrielescobar1')['items'][0]['id'])
print(spotify.get_playlist_items('6CKkmPw6rTdI0NRO4xnbhn')['items'][-1]['track']['artists'][0]['name'])
print(spotify.get_playlist_items('6CKkmPw6rTdI0NRO4xnbhn')['items'][-1]['track']['id'])

In [ ]:
my_playlists = UserPlaylists('darkness241100', client_id,client_secret)

average = my_playlists.playlist_audio_features('non')
'''
print("Danceability: " + str(average['danceability']))
print("Energy: " + str(average['energy']))
print("Loudness: " + str(average['loudness']))
print("Tempo: " + str(average['tempo']))

'''
print(average)



In [ ]:
my_profile = spotify.get_user_profile('gabrielescobar1')
print(my_profile)

In [ ]:
print(spotify.get_track(spotify.search_track_id('Dance', 'Drake')))
print(spotify.get_album('0UywfDKYlyiu1b38DRrzYD'))